In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [193]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 5
EVENT_NAME = 'EMA_CROSS_9_20'
LABEL_NAME = "TB"

START_DATE = "20240101"
END_DATE = "20241231"

RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "interm/events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATURES.pkl"
EVENT_FILE_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}.pkl"

In [194]:
DIRECTION_LABEL_DIR = BASE_DIR / "direction_labels"
DIRECTION_LABEL_DIR.mkdir(parents=True, exist_ok=True)
DIRECTION_LABEL_FILE_PATH = DIRECTION_LABEL_DIR / f"{RESAMPLED_NAME}-{EVENT_NAME}-{LABEL_NAME}.pkl"

In [195]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)
t_events = pd.read_pickle(EVENT_FILE_PATH)

CPU times: user 1.18 ms, sys: 7.16 ms, total: 8.35 ms
Wall time: 7.36 ms


In [196]:
df.loc[t_events].head()

,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,close_fd_return,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-05 05:00:00,144.8500,144.8675,144.8035,144.8095,3.210920e+12,0.006975,-0.000283,-0.0410,-0.000283,2.688681,...,5,1,9.659258e-01,2.588190e-01,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
2024-01-05 06:15:00,144.9190,144.9940,144.9135,144.9935,2.236320e+12,0.006505,0.000500,0.0725,0.000500,2.856262,...,5,1,1.000000e+00,6.123234e-17,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
2024-01-05 08:00:00,145.0775,145.0840,144.8535,144.8725,8.074380e+12,0.005622,-0.001423,-0.2065,-0.001424,2.535400,...,5,1,8.660254e-01,-5.000000e-01,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
2024-01-05 08:55:00,145.0755,145.1055,145.0580,145.0775,4.231930e+12,0.005028,0.000021,0.0030,0.000021,2.757985,...,5,1,8.660254e-01,-5.000000e-01,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
2024-01-05 12:15:00,145.2010,145.2515,145.1890,145.2065,2.724900e+12,0.005052,0.000034,0.0050,0.000034,2.727089,...,5,1,1.224647e-16,-1.000000e+00,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025


## Calculate Target Volatility (trgt)
This will be used for the calculation of dynamic TP/SL


In [197]:
import pandas_ta as ta

In [198]:
# 預計持單 1 小時 (60 minutes)
# 計算 1 小時內價格可能到哪裡
# 計算 1 小時 normalized atr
atr12 = df.ta.natr(length=12)

In [199]:
atr12

timestamp
2024-01-05 03:25:00         NaN
2024-01-05 03:30:00         NaN
2024-01-05 03:35:00         NaN
2024-01-05 03:40:00         NaN
2024-01-05 03:45:00         NaN
                         ...   
2024-12-30 23:35:00    0.024607
2024-12-30 23:40:00    0.024118
2024-12-30 23:45:00    0.025192
2024-12-30 23:50:00    0.024895
2024-12-30 23:55:00    0.024015
Name: NATR_12, Length: 73782, dtype: float64

In [200]:
trgt = atr12.reindex(t_events, method='ffill') / 100 # rate

In [201]:
(trgt).describe()

count    3368.000000
mean        0.000416
std         0.000287
min         0.000054
25%         0.000226
50%         0.000349
75%         0.000521
max         0.002815
Name: NATR_12, dtype: float64

## Apply Triple Barrier
We use daily volatility to calculate TP/SL, and we set our T1 as 1day

In [202]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd

def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    pt = ptSl[0] * events_['trgt'] if ptSl[0] > 0 else pd.Series(index=events.index)
    sl = -ptSl[1] * events_['trgt'] if ptSl[1] > 0 else pd.Series(index=events.index)

        
    for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns (rate)
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
        
    return out


def parallel_apply(func, items, num_threads=4, **kwargs):
    
    def worker(molecule):
        return func(molecule=molecule, **kwargs)

    chunks = np.array_split(items, num_threads)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return pd.concat(results).sort_index()


def getEvents(close, tEvents, ptSl, trgt, minTrgt, numThreads=4, t1=False, side=None):
    # Step 1: Filter targets
    trgt = trgt.loc[tEvents]
    trgt = trgt[trgt > minTrgt]

    # Step 2: Set vertical barrier (t1)
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)

    # Step 3: Build events DataFrame
    if side is None:
        side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
    else:
        side_, ptSl_ = side.loc[trgt.index], ptSl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])

    # Step 4: Apply barriers in parallel
    df0 = parallel_apply(
        func=applyPtSlOnT1,
        items=events.index,
        num_threads=numThreads,
        close=close,
        events=events,
        ptSl=ptSl_
    )

    # Step 5: Choose the first touched barrier
    events['t1'] = df0.dropna(how='all').min(axis=1)
    
    if side is None:
        events = events.drop('side', axis=1)
        
    return events


In [203]:
def get_vertical_barrier(t_events, close, delta=pd.Timedelta(minutes=60)):
    """
    For each event in t_events, find the timestamp in close.index 
    that is at least delta later. Returns a pd.Series of t1.
    """
    barrier_times = t_events + delta
    t1_idx = close.index.searchsorted(barrier_times)
    valid_idx = t1_idx[t1_idx < len(close)]
    t1 = pd.Series(close.index[valid_idx], index=t_events[:len(valid_idx)])
    return t1

In [204]:
t1 = get_vertical_barrier(t_events, df['close'], delta=pd.Timedelta(minutes=120))

In [205]:
t1

timestamp
2024-01-05 05:00:00   2024-01-05 07:00:00
2024-01-05 06:15:00   2024-01-05 08:15:00
2024-01-05 08:00:00   2024-01-05 10:00:00
2024-01-05 08:55:00   2024-01-05 10:55:00
2024-01-05 12:15:00   2024-01-05 14:15:00
                              ...        
2024-12-30 07:05:00   2024-12-30 09:05:00
2024-12-30 08:05:00   2024-12-30 10:05:00
2024-12-30 09:50:00   2024-12-30 11:50:00
2024-12-30 16:30:00   2024-12-30 18:30:00
2024-12-30 18:00:00   2024-12-30 20:00:00
Name: timestamp, Length: 3367, dtype: datetime64[ns]

In [206]:
trgt.describe()

count    3368.000000
mean        0.000416
std         0.000287
min         0.000054
25%         0.000226
50%         0.000349
75%         0.000521
max         0.002815
Name: NATR_12, dtype: float64

In [207]:
minTrgt = trgt.quantile(.3)
minTrgt
# minTrgt = 0

np.float64(0.00024668613711053774)

In [208]:
%%time

events = getEvents(
    close=df['close'],
    tEvents=t_events, 
    ptSl=[1,1], 
    trgt=trgt, 
    minTrgt=minTrgt, 
    numThreads=20, 
    t1=t1, 
    side=None
)

CPU times: user 564 ms, sys: 13.5 ms, total: 577 ms
Wall time: 568 ms


In [209]:
events.head()

,t1,trgt
timestamp,,
2024-01-05 05:00:00,2024-01-05 05:05:00,0.000286
2024-01-05 06:15:00,2024-01-05 06:20:00,0.000393
2024-01-05 08:00:00,2024-01-05 08:05:00,0.000669
2024-01-05 08:55:00,2024-01-05 09:55:00,0.000579
2024-01-05 12:15:00,2024-01-05 12:20:00,0.000349


In [210]:
events = events.dropna().copy()

# Keep only intraday events

In [211]:
events = events[events.index.date==events.t1.dt.date]

## Create classification labels

In [212]:
def getBins(events, close, minRet=0.0, t1=None):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    -t1 is original vertical barrier series
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    # 1) prices aligned with events
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # 2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[
        events_.index] - 1
    if 'side' in events_: out['ret'] *= events_['side']  # meta-labeling
    out['bin'] = np.sign(out['ret'])

    if 'side' not in events_:
        # only applies when not meta-labeling.
        # to update bin to 0 when vertical barrier is touched, we need the
        # original vertical barrier series since the events['t1'] is the time
        # of first touch of any barrier and not the vertical barrier
        # specifically. The index of the intersection of the vertical barrier
        # values and the events['t1'] values indicate which bin labels needs
        # to be turned to 0.
        vtouch_first_idx = events[events['t1'].isin(t1.values)].index
        out.loc[vtouch_first_idx, 'bin'] = 0.

    if 'side' in events_: out.loc[out['ret'] <= 0, 'bin'] = 0  # meta-labeling
    out.loc[out['ret'].abs() < minRet, 'bin'] = 0. # 不做獲利過少的交易 (設為 0)
    # out = out.loc[out['ret'].abs() >= minRet] # 過濾掉獲利過少的交易 (刪掉)
    return out

In [213]:
labels = getBins(events, df['close'], minRet=0, t1=t1)

In [214]:
print(labels.ret.abs().describe())
print(labels.ret.abs().quantile(.3))

count    2327.000000
mean        0.000745
std         0.000613
min         0.000007
25%         0.000434
50%         0.000591
75%         0.000844
max         0.008783
Name: ret, dtype: float64
0.00046178823489584353


In [215]:
labels.bin.value_counts()

bin
 1.0    1144
-1.0    1030
 0.0     153
Name: count, dtype: int64

In [216]:
labels["bin_class"] = labels["bin"] + 1

In [217]:
labeled_events = events.join(labels, how='inner')

In [218]:
labeled_events.head()

,t1,trgt,ret,bin,bin_class
timestamp,,,,,
2024-01-05 05:00:00,2024-01-05 05:05:00,0.000286,0.000411,1.0,2.0
2024-01-05 06:15:00,2024-01-05 06:20:00,0.000393,0.000714,1.0,2.0
2024-01-05 08:00:00,2024-01-05 08:05:00,0.000669,0.000984,1.0,2.0
2024-01-05 08:55:00,2024-01-05 09:55:00,0.000579,0.000748,1.0,2.0
2024-01-05 12:15:00,2024-01-05 12:20:00,0.000349,-0.000351,-1.0,0.0


## Save events to disk

In [219]:
labeled_events.to_pickle(DIRECTION_LABEL_FILE_PATH)

In [220]:
DIRECTION_LABEL_FILE_PATH

PosixPath('../data/direction_labels/USDJPY-5m-20240101-20241231-EMA_CROSS_9_20-TB.pkl')